Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import os
import time
import random as rd
import math

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (350000, 28, 28) (350000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (350000, 784) (350000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Function running selected model:

In [5]:
def run_fit(graph, regularization_list, num_steps, print_every_n,train_dataset,train_labels):
    for r in regularization_list:
        with tf.Session(graph=graph) as session:
            tf.initialize_all_variables().run()
            session.run(regularization_parameter.assign(r))
            print("---------------------------------------------------------")
            print("Initialized - r = ", regularization_parameter.eval())
            for step in range(num_steps):
                # Pick an offset within the training data, which has been randomized.
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = train_dataset[offset:(offset + batch_size), :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                # Prepare a dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
                _, l, predictions = session.run(
                  [optimizer, loss, train_prediction], feed_dict=feed_dict)
                if (step % print_every_n == 0):
                  print("Minibatch loss at step %d: %f" % (step, l))
                  print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                  print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Creating and running graph for logistic regression:

In [7]:
batch_size = 128

graphLogReg = tf.Graph()
with graphLogReg.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  regularization_parameter = tf.Variable(0.0, trainable = False)
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+regularization_parameter*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  run_fit(graphLogReg, [1, .5, .05, 5e-3, 7.5e-4, 5e-4, 2.5e-4, 5e-5, 0], 30001, 10000,train_dataset,train_labels) 

---------------------------------------------------------
Initialized - r =  1.0
Minibatch loss at step 0: 3039.232666
Minibatch accuracy: 12.5%
Validation accuracy: 15.5%
Minibatch loss at step 10000: 2.079980
Minibatch accuracy: 48.4%
Validation accuracy: 48.6%
Minibatch loss at step 20000: 2.377190
Minibatch accuracy: 42.2%
Validation accuracy: 45.3%
Minibatch loss at step 30000: 2.079784
Minibatch accuracy: 37.5%
Validation accuracy: 51.0%
Test accuracy: 55.6%
---------------------------------------------------------
Initialized - r =  0.5
Minibatch loss at step 0: 1520.993530
Minibatch accuracy: 7.8%
Validation accuracy: 12.8%
Minibatch loss at step 10000: 1.671805
Minibatch accuracy: 64.1%
Validation accuracy: 63.1%
Minibatch loss at step 20000: 1.587291
Minibatch accuracy: 63.3%
Validation accuracy: 64.5%
Minibatch loss at step 30000: 1.688589
Minibatch accuracy: 62.5%
Validation accuracy: 65.4%
Test accuracy: 71.3%
---------------------------------------------------------
Initi

 5e-5 seems to be the best. But the improvement comparing to not using regularization is small probably because there is a lot of training data and model is not very complex.

Creating and running graph for neural network:

In [11]:
batch_size = 128
hidden_nodes = 1024
graphNN = tf.Graph()
with graphNN.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

      # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=1 / math.sqrt(float(image_size * image_size))))
  hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)

  weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  regularization_parameter = tf.Variable(0.0, trainable = False)
    # Training computation.
  logits = tf.matmul(hidden, weights) + biases
 
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+ \
    regularization_parameter*(tf.nn.l2_loss(weights)+tf.nn.l2_loss(hidden_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)
  run_fit(graphNN, [ .5, .05, 5e-4,  0], 5001, 2000,train_dataset,train_labels)   

---------------------------------------------------------
Initialized - r =  0.5
Minibatch loss at step 0: 2202.272461
Minibatch accuracy: 11.7%
Validation accuracy: 40.9%
Minibatch loss at step 2000: 2.240600
Minibatch accuracy: 38.3%
Validation accuracy: 39.4%
Minibatch loss at step 4000: 2.235852
Minibatch accuracy: 46.1%
Validation accuracy: 45.4%
Test accuracy: 45.4%
---------------------------------------------------------
Initialized - r =  0.05
Minibatch loss at step 0: 234.405441
Minibatch accuracy: 9.4%
Validation accuracy: 38.1%
Minibatch loss at step 2000: 1.042582
Minibatch accuracy: 84.4%
Validation accuracy: 77.4%
Minibatch loss at step 4000: 1.002550
Minibatch accuracy: 83.6%
Validation accuracy: 78.1%
Test accuracy: 86.3%
---------------------------------------------------------
Initialized - r =  0.0005
Minibatch loss at step 0: 17.164541
Minibatch accuracy: 10.2%
Validation accuracy: 44.3%
Minibatch loss at step 2000: 0.899469
Minibatch accuracy: 89.8%
Validation acc

Probably I need train the network for longer to see some effects of regularization.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [14]:
 run_fit(graphNN, [ 0 ], 10001, 10000,train_dataset[:batch_size*3,:],train_labels[:batch_size*3,:])

---------------------------------------------------------
Initialized - r =  0.0
Minibatch loss at step 0: 10.023168
Minibatch accuracy: 9.4%
Validation accuracy: 36.5%
Minibatch loss at step 10000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Test accuracy: 80.5%


100% fit to training data, drop in accuracy on test and validation data.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [15]:
batch_size = 128
hidden_nodes = 1024
graphNNdropout = tf.Graph()
with graphNNdropout.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
      # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=1 / math.sqrt(float(image_size * image_size))))
  hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  hidden_after_drop = tf.nn.dropout(hidden, 0.5)
  weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  regularization_parameter = tf.Variable(0.0, trainable = False)
    # Training computation.
  logits = tf.matmul(hidden_after_drop, weights) + biases
 
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+regularization_parameter*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)

In [16]:
run_fit(graphNNdropout, [0], 10001, 10000,train_dataset[:batch_size*3,:],train_labels[:batch_size*3,:])

---------------------------------------------------------
Initialized - r =  0.0
Minibatch loss at step 0: 16.497303
Minibatch accuracy: 12.5%
Validation accuracy: 43.1%
Minibatch loss at step 10000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.3%
Test accuracy: 82.8%


Improvement in test accuracy.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [18]:
 def file_name(epoch_nb,test_acc_log, num_epochs, batch_size,hidden_nodes, learning_rate, decay_rate, \
              dropout_keep_prob, l2_penalty, train_dataset,train_labels, path_to_save):
    file_name = str(epoch_nb)+'_'+str(test_acc_log) + '_ne.'+str(num_epochs)+'_bs.' + str(batch_size)+ \
    '_lr.'+str(learning_rate) + '_dr'+str(decay_rate) + '_d.' + str(dropout_keep_prob) + \
    '_l2.'+ str(l2_penalty) + '_t'+'.csv'
    file_name = os.path.join(path_to_save, file_name)
    return file_name


In [19]:
def run_model(num_epochs, batch_size,hidden_nodes, learning_rate, decay_rate, dropout_keep_prob, l2_penalty, \
                train_dataset,train_labels, path_to_save, quiet = True, epoch_calc = False, beta1=0.9, beta2=0.999):
    num_steps = (train_labels.shape[0] * num_epochs) / batch_size
    epoch_log = np.full(num_steps, np.nan)
    loss_log = np.full(num_steps, np.nan)
    acc_log = np.full(num_steps, np.nan)
    val_acc_log = np.full(num_steps, np.nan)
    test_acc_log = np.full(num_steps, np.nan)
    print(num_steps)
    if not os.path.exists(path_to_save):
        os.makedirs(path_to_save)
        
    epoch_nb = -1
    graph = tf.Graph()
    # GRAPH
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.

        tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        # Input parameters:
        tf_learning_rate_0 = tf.placeholder(tf.float32)
        tf_dropout_keep_prob =  tf.placeholder(tf.float32)
        tf_l2_penalty =  tf.placeholder(tf.float32)
        tf_decay_rate = tf.placeholder(tf.float32)
        tf_step_nb = tf.placeholder(tf.float32)
        # Variables.
        # 1 layer      
        hidden_weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, 2048], stddev=2 / math.sqrt(float(image_size * image_size))))
        hidden_biases = tf.Variable(tf.zeros([2048]))
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
        hidden_after_drop = tf.nn.dropout(hidden, tf_dropout_keep_prob)

        # 2 layer 
        hidden_weights_2 = tf.Variable(
        tf.truncated_normal([2048, hidden_nodes], stddev=2 / math.sqrt(float(hidden_nodes))))
        hidden_biases_2 = tf.Variable(tf.zeros([hidden_nodes]))
        hidden_2 = tf.nn.relu(tf.matmul(hidden_after_drop, hidden_weights_2) + hidden_biases_2)
        hidden_after_drop_2 = tf.nn.dropout(hidden_2, tf_dropout_keep_prob)

        # 3 layer 
        hidden_weights_3 = tf.Variable(
        tf.truncated_normal([hidden_nodes, hidden_nodes], stddev=2 / math.sqrt(float(hidden_nodes))))
        hidden_biases_3 = tf.Variable(tf.zeros([hidden_nodes]))
        hidden_3 = tf.nn.relu(tf.matmul(hidden_after_drop_2, hidden_weights_3) + hidden_biases_3)
        hidden_after_drop_3 = tf.nn.dropout(hidden_3, tf_dropout_keep_prob)

        weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits = tf.matmul(hidden_after_drop_3, weights) + biases
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        
        #reg = tf_l2_penalty*(tf.nn.l2_loss(hidden_weights)+ 
        #                     tf.nn.l2_loss(hidden_weights_2)+ 
        #                     tf.nn.l2_loss(hidden_weights_3)+
        #                     tf.nn.l2_loss(weights))
        #loss+=reg
        # Optimizer.
        tf_learning_rate = tf.train.exponential_decay(tf_learning_rate_0, tf_step_nb, 1000, tf_decay_rate)
        optimizer = tf.train.AdamOptimizer(tf_learning_rate_0,beta1,beta2).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(

                  tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases),

                                        hidden_weights_2) + hidden_biases_2),

                                        hidden_weights_3) + hidden_biases_3), weights) + biases)

        test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(
        tf.nn.relu(tf.matmul(
        tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases),
                                hidden_weights_2) + hidden_biases_2),
                                hidden_weights_3) + hidden_biases_3), weights) + biases)
        
 #starting new session 
    epoch_nb = -1
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                        tf_learning_rate_0 : learning_rate, tf_dropout_keep_prob : dropout_keep_prob, 
                        tf_l2_penalty : l2_penalty,tf_decay_rate : decay_rate, tf_step_nb : step}
            _, l, predictions = session.run(
                    [optimizer, loss, train_prediction], feed_dict=feed_dict)
            loss_log[step] = l
            epoch_log[step] = step * batch_size / float(train_labels.shape[0])
            if (step % 50 == 0):
                val_acc_log[step] = accuracy(valid_prediction.eval(), valid_labels)
                acc_log[step] = accuracy(predictions, batch_labels)
                if not quiet: 
                    print("Minibatch loss at step %d: %f" % (step, l))
                    print("Minibatch accuracy: %.1f%%" % acc_log[step])
                    print("Validation accuracy: %.1f%%" % val_acc_log[step])
            if epoch_calc and epoch_log[step] > epoch_nb:
                epoch_nb+=1
                test_acc_log[step] = accuracy(test_prediction.eval(), test_labels)
                val_acc_log[step] = accuracy(valid_prediction.eval(), valid_labels)
                acc_log[step] = accuracy(predictions, batch_labels)
                print("Test accuracy: %.1f%%" %  test_acc_log[step])
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % acc_log[step])
                print("Validation accuracy: %.1f%%" % val_acc_log[step])
                fn = file_name(epoch_nb,test_acc_log[step], num_epochs, batch_size,hidden_nodes, \
                               learning_rate, decay_rate, dropout_keep_prob, l2_penalty, \
                               train_dataset,train_labels, path_to_save)
                np.savetxt(fn, np.vstack((epoch_log, loss_log, acc_log, val_acc_log, test_acc_log)).T, 
                   delimiter = ';', fmt='%.10f')
        
        test_acc_log[step] = accuracy(test_prediction.eval(), test_labels)
        fn = file_name(epoch_nb,test_acc_log[step], num_epochs, batch_size,hidden_nodes, \
                       learning_rate, decay_rate, dropout_keep_prob, l2_penalty, \
                       train_dataset,train_labels, path_to_save)
        np.savetxt(fn, np.vstack((epoch_log, loss_log, acc_log, val_acc_log, test_acc_log)).T, 
                   delimiter = ';', fmt='%.10f')
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Test if we get 100% accuracy on small training dataset:

In [20]:
run_model(300, 128,1024, 0.005, 0.98, 1.0, 0, train_dataset[:256],train_labels[:256], '/notebooks/run1', False)

600
Minibatch loss at step 0: 59.842236
Minibatch accuracy: 12.5%
Validation accuracy: 24.4%
Minibatch loss at step 50: 0.000042
Minibatch accuracy: 100.0%
Validation accuracy: 63.5%
Minibatch loss at step 100: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.3%
Minibatch loss at step 150: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.3%
Minibatch loss at step 200: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.3%
Minibatch loss at step 250: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.4%
Minibatch loss at step 300: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Minibatch loss at step 350: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Minibatch loss at step 400: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Minibatch loss at step 450: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.6%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.6

OK!

Testing parameters:

In [ ]:
num_epochs = 2
hidden_nodes = 1024
for i in range(50):
    t0 = time.time()
    batch_size = 1024
    learning_rate = 10 ** rd.uniform(-5., 1)
    decay_rate = rd.uniform(0.9,1)
    dropout_keep_prob = rd.uniform(0.4,1)
    l2_penalty = 10 ** rd.uniform(-6., 1.)
    print('Model',i,'batch_size:',batch_size, 'learning_rate:', learning_rate,\
          'decay_rate:', decay_rate, 'dropout_keep_prob:', dropout_keep_prob,
         'l2_penalty:', l2_penalty)
    run_model(num_epochs, batch_size,hidden_nodes, learning_rate,decay_rate, dropout_keep_prob, l2_penalty, \
                train_dataset,train_labels, '/notebooks/run1', True, True)   
    t1 = time.time()
    print('Model',i,'- time:',(t1-t0)/60, 'min.')

In [24]:
num_epochs = 50
decay_rate = 0.99
batch_size = 1024
learning_rate = 0.001
dropout_keep_prob = 0.9
#l2_penalty = 1.08
run_model(num_epochs, batch_size,hidden_nodes, learning_rate,decay_rate, dropout_keep_prob, 0, \
                train_dataset,train_labels, '/notebooks/run2', True, True)


17089
Test accuracy: 37.2%
Minibatch loss at step 0: 54.744087
Minibatch accuracy: 9.3%
Validation accuracy: 34.2%
Test accuracy: 44.1%
Minibatch loss at step 1: 99.426155
Minibatch accuracy: 30.0%
Validation accuracy: 39.1%
Test accuracy: 91.0%
Minibatch loss at step 342: 0.614311
Minibatch accuracy: 81.5%
Validation accuracy: 84.7%
Test accuracy: 92.8%
Minibatch loss at step 684: 0.434510
Minibatch accuracy: 86.9%
Validation accuracy: 86.9%
Test accuracy: 93.5%
Minibatch loss at step 1026: 0.388294
Minibatch accuracy: 88.2%
Validation accuracy: 87.6%
Test accuracy: 94.1%
Minibatch loss at step 1368: 0.320395
Minibatch accuracy: 89.9%
Validation accuracy: 88.6%
Test accuracy: 94.5%
Minibatch loss at step 1709: 0.339919
Minibatch accuracy: 89.2%
Validation accuracy: 89.0%
Test accuracy: 94.7%
Minibatch loss at step 2051: 0.305903
Minibatch accuracy: 90.4%
Validation accuracy: 89.2%
Test accuracy: 94.8%
Minibatch loss at step 2393: 0.299752
Minibatch accuracy: 90.5%
Validation accuracy:

It a bit overfitted.